In [23]:

import numpy as np
import cv2
import sklearn.model_selection as sk
from analysis import parse_protobufs
from ipdb import set_trace as debug

class struct():
	pass

def parse_data(save=True):
    path = struct()
    path.data_folder = 'TeleOpVRSession_2018-02-05_15-44-11/'
    path.data_name = '_SessionStateData.proto'
    data = parse_protobufs(path)
    print(len(data.states))	
    # example data extraction of x value of object/item 0 in training example 0: data.states[0].items[0].x
    num_examples = len(data.states) # number or screenshots
    num_items = []  # number of items in each example
    labels = []
    X_rgb = np.empty([0,299,299,3])
    X_d = np.empty([0,299,299])
    # format labels into n x 6 array
    for i in range(10):
        print(i)
        num_items.append(len(data.states[i].items))
        img_name = str(data.states[i].snapshot.name)
        depth_name = img_name[:-4] + '-Depth.jpg'

        # read in rgb and depth images and add a new axis to them to indicate which snapshot index for each image
        rgb_img = np.expand_dims(cv2.imread(img_name, 1), axis=0)
        depth_img = np.expand_dims(cv2.imread(depth_name, 0), axis = 0)

        for j in range(num_items[i]):
            # input data (X)
            X_rgb = np.vstack([X_rgb, rgb_img])
            X_d = np.vstack([X_d, depth_img])

            # Output label (Y)
            rlabel = data.states[i].items[j] 
            print(rlabel.id)
            current_label = [data.states[i].snapshot.name,rlabel.id, rlabel.x, rlabel.y, rlabel.z, rlabel.roll, rlabel.pitch, rlabel.yaw]
            labels.append(current_label) 

    # convert to numpy array
    y = np.array(labels)

    if save:
        save_path = 'data/'
        np.save(save_path + "X_rgb.npy", X_rgb)
        np.save(save_path + "X_d.npy", X_d)
        np.save(save_path + "y.npy", y)

    return X_rgb, X_d, y


if __name__ == '__main__':
    X_rgb, X_d, y = parse_data(save=True)
    X = (np.concatenate((X_rgb,np.expand_dims(X_d, axis=3)), axis=3))
    X_train, X_test, y_train, y_test = sk.train_test_split(X,y,test_size=.3, random_state=42)	# random_state=42 ensure indices are same for train/test set for X_rgb and X_d since they must match



10000
0
14
37
27
47
33
30
11
53
24
9
7
19
13
10
43
48
1
20
4
53
35
13
41
16
30
0
44
47
24
52
40
10
1
21
37
19
2
39
47
12
21
30
15
7
34
36
14
27
53
3
29
10
0
4
16
42
53
13
33
36
15
39
30
47
20
18
44
38
14
4
17
4
8
36
28
9
5
22
1
54
15
7
11
6
29
8
7
24
16
6
36
7
2
10
30
27
6
28
54
4
40
36
45
39
35
15
0
47
8
33
2
19
35
14
29
40
21
9
45
5
18
32
9
24
14


In [21]:
# let's try opening one of these label files 
label = np.load('./data/y.npy')
print(label)

[['TeleOpVRSession_2018-02-05_15-44-11/26dc3d94-03a2-4c1e-a871-308188bac059_0.jpg'
  '-0.5373133420944214' '0.48769885301589966' '1.0154430866241455'
  '142.422607421875' '-144.23684692382812' '-150.07513427734375']
 ['TeleOpVRSession_2018-02-05_15-44-11/26dc3d94-03a2-4c1e-a871-308188bac059_0.jpg'
  '0.5068195462226868' '1.0124906301498413' '1.4353491067886353'
  '-36.417240142822266' '-88.56159973144531' '-28.581396102905273']
 ['TeleOpVRSession_2018-02-05_15-44-11/26dc3d94-03a2-4c1e-a871-308188bac059_0.jpg'
  '0.3202996850013733' '0.43912041187286377' '0.976573646068573'
  '-13.610997200012207' '-84.53562927246094' '58.7875862121582']
 ['TeleOpVRSession_2018-02-05_15-44-11/26dc3d94-03a2-4c1e-a871-308188bac059_0.jpg'
  '-0.002812027931213379' '1.0395092964172363' '1.4569677114486694'
  '35.35747146606445' '-53.61668395996094' '73.13513946533203']
 ['TeleOpVRSession_2018-02-05_15-44-11/26dc3d94-03a2-4c1e-a871-308188bac059_0.jpg'
  '-1.6668826341629028' '0.7202595472335815' '1.201523900